In [13]:
from dotenv import load_dotenv

_ = load_dotenv()

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [15]:
tool = TavilySearchResults(max_results=2)

In [16]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
# pip install langgraph-checkpoint-sqlite
from langgraph.checkpoint.sqlite import SqliteSaver

# memory = SqliteSaver.from_conn_string(":memory:")

In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # Compile the graph with the checkpointer
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [19]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o-mini")

with SqliteSaver.from_conn_string("checkpoints.sqlite") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)

    messages = [HumanMessage(content="What is the weather in Sydney?")]
    thread = {"configurable": {"thread_id": "1"}}

    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])


    messages = [HumanMessage(content="How about in Melbourne?")]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

    messages = [HumanMessage(content="Which one is warmer?")]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

    snapshot = abot.graph.get_state(thread)
    print(snapshot.values.keys())

    messages = snapshot.values["messages"]

    for i, msg in enumerate(messages):
        print(f"[{i}] {msg.type.upper()} - {msg.content}")

    messages = [HumanMessage(content="Which one is colder?")]
    thread = {"configurable": {"thread_id": "2"}} # change to a different thread_id
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_v1c8uwfPtjo36c73cSCMSKJq', 'function': {'arguments': '{"query":"current weather report Sydney"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 4261, 'total_tokens': 4282, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4224}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Bw5DHdJJDt5UoUhyvQDknfROmPZcQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--914a6123-e27a-460e-8232-19c6dcb70810-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather report Sydney'}, 'id': 'call_v1c8uwfPtjo36c73cSCMSKJq', 'type': 'tool_call'}], usage_metadata={'input_tokens':

In [20]:
with SqliteSaver.from_conn_string("checkpoints.sqlite") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)

    messages = [HumanMessage(content="Which one is colder?")]
    thread = {"configurable": {"thread_id": "1"}}

    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

[AIMessage(content='Currently, both Sydney and Melbourne have similar temperatures of around 8°C (46°F). Therefore, neither city is noticeably colder than the other at this moment; they are quite comparable in temperature.\n\nTypically, however, Melbourne is often colder than Sydney, especially during the winter months. Should you need more details or information, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 5549, 'total_tokens': 5619, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 5504}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Bw5Dby10qD0h3657IaMV9hhqqodXk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0385590-aa96-4011-93be-f6badd5aab12-0', usage_metadata={'input_to

In [21]:
with SqliteSaver.from_conn_string("checkpoints.sqlite") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)

    messages = [HumanMessage(content="Which one is bigger?")]
    thread = {"configurable": {"thread_id": "1"}}

    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

[AIMessage(content="Sydney is larger than Melbourne in terms of both population and metropolitan area.\n\n- **Population**: \n  - **Sydney** has a population of over 5.3 million people, making it the most populous city in Australia.\n  - **Melbourne** has a population of about 5 million people, making it the second-most populous city in the country.\n\n- **Area**:\n  - **Sydney's metropolitan area** covers approximately 12,368 square kilometers (4,775 square miles).\n  - **Melbourne's metropolitan area** spans about 9,992 square kilometers (3,858 square miles).\n\nIn summary, Sydney is larger than Melbourne both in terms of population and geographic area. If you have further questions or need additional information, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 5631, 'total_tokens': 5789, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejec

In [22]:
import sqlite3

conn = sqlite3.connect("checkpoints.sqlite")
cursor = conn.cursor()

import pandas as pd

def show_query_as_df(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [description[0] for description in cursor.description]
    return pd.DataFrame(rows, columns=columns)

# 查看 checkpoints 表结构
df_checkpoints_schema = show_query_as_df(cursor, "PRAGMA table_info(checkpoints);")
display(df_checkpoints_schema)

# 查看 writes 表结构
df_writes_schema = show_query_as_df(cursor, "PRAGMA table_info(writes);")
display(df_writes_schema)

# 查看 checkpoints 表数据
df_checkpoints_data = show_query_as_df(cursor, "SELECT * FROM checkpoints;")
display(df_checkpoints_data)

# 查看 writes 表数据
df_writes_data = show_query_as_df(cursor, "SELECT * FROM writes;")
display(df_writes_data)

,cid,name,type,notnull,dflt_value,pk
0,0,thread_id,TEXT,1,None,1
1,1,checkpoint_ns,TEXT,1,'',2
2,2,checkpoint_id,TEXT,1,None,3
3,3,parent_checkpoint_id,TEXT,0,None,0
4,4,type,TEXT,0,None,0
5,5,checkpoint,BLOB,0,None,0
6,6,metadata,BLOB,0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,thread_id,TEXT,1,None,1
1,1,checkpoint_ns,TEXT,1,'',2
2,2,checkpoint_id,TEXT,1,None,3
3,3,task_id,TEXT,1,None,4
4,4,idx,INTEGER,1,None,5
5,5,channel,TEXT,1,None,0
6,6,type,TEXT,0,None,0
7,7,value,BLOB,0,None,0


,thread_id,checkpoint_ns,checkpoint_id,parent_checkpoint_id,type,checkpoint,metadata
0,1,,1f066ec4-49c7-6046-bfff-aa65c5cbcec3,None,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:08:59....,"b'{""source"": ""input"", ""step"": -1, ""parents"": {}}'"
1,1,,1f066ec4-49c8-66a8-8000-4b08d16125ef,1f066ec4-49c7-6046-bfff-aa65c5cbcec3,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:08:59....,"b'{""source"": ""loop"", ""step"": 0, ""parents"": {}}'"
2,1,,1f066ec4-6070-60b8-8001-00e909555671,1f066ec4-49c8-66a8-8000-4b08d16125ef,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:02....,"b'{""source"": ""loop"", ""step"": 1, ""parents"": {}}'"
3,1,,1f066ec4-97be-65c4-8002-8daa640e6861,1f066ec4-6070-60b8-8001-00e909555671,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:07....,"b'{""source"": ""loop"", ""step"": 2, ""parents"": {}}'"
4,1,,1f066ec4-a2af-6910-8003-1170cfbe1fe8,1f066ec4-97be-65c4-8002-8daa640e6861,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:09....,"b'{""source"": ""loop"", ""step"": 3, ""parents"": {}}'"
5,1,,1f066ec4-deff-6b18-8004-ef6df0779f6a,1f066ec4-a2af-6910-8003-1170cfbe1fe8,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:15....,"b'{""source"": ""loop"", ""step"": 4, ""parents"": {}}'"
6,1,,1f066ec4-fd4c-638c-8005-cc73c01fda20,1f066ec4-deff-6b18-8004-ef6df0779f6a,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:18....,"b'{""source"": ""loop"", ""step"": 5, ""parents"": {}}'"
7,1,,1f066ec4-fd55-69a0-8006-d0009df1319f,1f066ec4-fd4c-638c-8005-cc73c01fda20,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:18....,"b'{""source"": ""input"", ""step"": 6, ""parents"": {}}'"
8,1,,1f066ec4-fd57-6480-8007-227b38662929,1f066ec4-fd55-69a0-8006-d0009df1319f,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:18....,"b'{""source"": ""loop"", ""step"": 7, ""parents"": {}}'"
9,1,,1f066ec5-09cc-6760-8008-07515c154ab5,1f066ec4-fd57-6480-8007-227b38662929,msgpack,b'\x86\xa1v\x04\xa2ts\xd9 2025-07-22T11:09:19....,"b'{""source"": ""loop"", ""step"": 8, ""parents"": {}}'"


,thread_id,checkpoint_ns,checkpoint_id,task_id,idx,channel,type,value
0,1,,1f066ec4-49c7-6046-bfff-aa65c5cbcec3,32a14325-9c75-400b-9283-1f51a401f863,0,messages,msgpack,b'\x91\xc7\xac\x05\x94\xbdlangchain_core.messa...
1,1,,1f066ec4-49c7-6046-bfff-aa65c5cbcec3,32a14325-9c75-400b-9283-1f51a401f863,1,branch:to:llm,null,b''
2,1,,1f066ec4-49c8-66a8-8000-4b08d16125ef,74937027-0a4e-d734-2697-0daf5bbe46de,0,messages,msgpack,b'\x91\xc8\x03\xf4\x05\x94\xbalangchain_core.m...
3,1,,1f066ec4-49c8-66a8-8000-4b08d16125ef,74937027-0a4e-d734-2697-0daf5bbe46de,1,branch:to:action,null,b''
4,1,,1f066ec4-6070-60b8-8001-00e909555671,204008f9-5db7-21cf-b262-300d3a722e9d,0,messages,msgpack,"b""\x91\xc8\x0c9\x05\x94\xbclangchain_core.mess..."
5,1,,1f066ec4-6070-60b8-8001-00e909555671,204008f9-5db7-21cf-b262-300d3a722e9d,1,branch:to:llm,null,b''
6,1,,1f066ec4-97be-65c4-8002-8daa640e6861,76e53e09-b1e4-1230-7924-e1743e7fbd87,0,messages,msgpack,b'\x91\xc8\x03\xf2\x05\x94\xbalangchain_core.m...
7,1,,1f066ec4-97be-65c4-8002-8daa640e6861,76e53e09-b1e4-1230-7924-e1743e7fbd87,1,branch:to:action,null,b''
8,1,,1f066ec4-a2af-6910-8003-1170cfbe1fe8,c79ddf42-7512-b309-c134-100060b4fc00,0,messages,msgpack,"b""\x91\xc8\x0c\x1c\x05\x94\xbclangchain_core.m..."
9,1,,1f066ec4-a2af-6910-8003-1170cfbe1fe8,c79ddf42-7512-b309-c134-100060b4fc00,1,branch:to:llm,null,b''


## Streaming tokens


In [32]:
# pip install aiosqlite
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [35]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}

async def run_stream():
    async with AsyncSqliteSaver.from_conn_string(":memory:") as memory:  # ✅ 注意这里是 async with
        abot = Agent(model, [tool], system=prompt, checkpointer=memory)

        async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
            kind = event["event"]
            if kind == "on_chat_model_stream":
                content = event["data"]["chunk"].content
                if content:
                    print(content, end="|")

await run_stream()


/Users/isdance/Desktop/ai_projects/agent_in_langgraph/.venv/lib/python3.12/site-packages/pydantic/_internal/_known_annotated_metadata.py:106: RuntimeWarning: coroutine 'run_stream' was never awaited
  def expand_grouped_metadata(annotations: Iterable[Any]) -> Iterable[Any]:


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_w7CLUrHdBKvW6jpLaVU4Krx9', 'type': 'tool_call'}
Back to the model!
The| current| weather| in| San| Francisco| is| mainly| cloudy|,| with| a| high| of| around| |64|°F| (|approximately| |18|°C|)| during| the| day|.| The| temperatures| are| expected| to| drop| to| a| low| of| about| |56|°F| (|approximately| |13|°C|)| at| night|.| Winds| are| coming| from| the| west| at| |10| to| |20| mph|.

|For| more| details|,| you| can| check| the| comprehensive| weather| report| [|here|](|https|://|weather|.com|/weather|/t|oday|/l|/S|an|+|Franc|isco|+|CA|+|US|CA|098|7|:|1|:|US|).|